<a href="https://colab.research.google.com/github/asymptotebag/covid-sentiment-polarity/blob/master/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Token Sentiment Analysis of COVID-19 Topic Tweets


##Preliminary Model/Baseline Results

In [ ]:
import os
data_root = '/data/'

Mounted at /content/drive


## Initial Data pre-processing

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import sklearn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
!pip install tweet-preprocessor
!pip install transformers
import preprocessor as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.4 MB/s 
     |████████████████████████████████| 182 kB 63.2 MB/s 


In [ ]:
# load the data
datasets = ['tweets_batch_0.csv', 
            'tweets_batch_1.csv', 
            'tweets_batch_2.csv', 
            'tweets_batch_3.csv']#['tweets_with_sentiment_small.csv'] #'100k_tweets_with_sentiment_355.csv'] 

df_stack = []
for idx, dataset_name in enumerate(datasets):
    df_stack.append(pd.read_csv(os.path.join(data_root, dataset_name), index_col=0))
    print('The length of dataset {} is {} tweets'.format(dataset_name, len(df_stack[idx])))

df_tweets = pd.concat(df_stack)
df_tweets.head(2)

The length of dataset tweets_batch_0.csv is 12410 tweets
The length of dataset tweets_batch_1.csv is 66992 tweets
The length of dataset tweets_batch_2.csv is 9526 tweets
The length of dataset tweets_batch_3.csv is 43667 tweets


,id,text,sentiment
0,1.342326e+18,We wish a grand success for Supreme Hero @IamS...,0.278125
1,1.342326e+18,"Sorry to disappoint, but I’ve not made a dolla...",-0.750000


In [ ]:
df_tweets = df_tweets.drop_duplicates(subset=['text'])
duplicate = df_tweets[df_tweets['text'].duplicated()]
print('Duplicates remaining: {}'.format(len(duplicate)))
print('Dataset length: {}'.format(len(df_tweets)))

Duplicates remaining: 0
Dataset length: 91481


In [ ]:
import spacy
nlp_model = spacy.load('en_core_web_sm')

# process the merged dataset
df_tweets[['tweet', 'sentiment']] = df_tweets[['text', 'sentiment']] # restrict to sentiment and tweet 
df_tweets = df_tweets.dropna(subset=['sentiment'])

# process text
df_tweets['text'] = df_tweets['text'].apply(lambda x: str(x)) # object to string
df_tweets['text'] = df_tweets['text'].apply(lambda x: x.lower()) # lower case text
df_tweets['text'] = df_tweets['text'].apply(p.clean)  # text - remove references, links, and hastaged text - https://pypi.org/project/tweet-preprocessor/

# hacky way to tokenize
tweet_tokens = df_tweets['text'].to_numpy()
txt = []
for doc in nlp_model.pipe(tweet_tokens, batch_size=32, n_process=3, disable=["parser", "ner"]):
    txt.append(' '.join([tok.lemma_ for tok in doc]))
df_tweets['text'] = txt

df_tweets['text'] = df_tweets['text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))  # remove punctuation
df_tweets['text'] = df_tweets['text'].apply(lambda x: re.sub(' +', ' ', x)) # remove redundant spaces

df_tweets.head()

<ipython-input-12-e6d4d48d9c95>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['text'] = df_tweets['text'].apply(lambda x: str(x)) # object to string
<ipython-input-12-e6d4d48d9c95>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['text'] = df_tweets['text'].apply(lambda x: x.lower()) # lower case text


,id,text,sentiment,tweet
0,1.342326e+18,we wish a grand success for supreme hero for r...,0.278125,We wish a grand success for Supreme Hero @IamS...
1,1.342326e+18,sorry to disappoint but I ve not make a dollar...,-0.750000,"Sorry to disappoint, but I’ve not made a dolla..."
2,1.342326e+18,this be wild the who change the definition of ...,0.237500,This is wild.\nThe WHO changed the definition ...
3,1.342326e+18,patch she be with us year this week we all wis...,0.600000,@mayawiley Patches. She is with us 2 years thi...
4,1.342326e+18,docsend ceo peer into a post pandemic future,0.000000,DocSend CEO peers into a post-pandemic future ...


In [ ]:
duplicate = df_tweets[df_tweets['text'].duplicated()]
print('Duplicates remaining: {}'.format(len(duplicate)))

Duplicates remaining: 6251


In [ ]:
df_tweets = df_tweets.drop_duplicates(subset=['text'])
duplicate = df_tweets[df_tweets['text'].duplicated()]
duplicate

,id,text,sentiment,tweet


In [ ]:
len(df_tweets)

85228

In [ ]:
# preprocess sentiment
def binarize(value):
    if value <= 1/2:
        return 0
    return 1

max_val = df_tweets['sentiment'].max()
min_val = df_tweets['sentiment'].min()
# normalize between to [0,1]
df_tweets['norm_sentiment'] = df_tweets['sentiment'].apply(lambda x: (x - min_val)/(max_val - min_val))

# binary sentiment
df_tweets['binary_sentiment'] = df_tweets['norm_sentiment'].apply(binarize)

df_tweets.head()

<ipython-input-20-e11ddfafc2ca>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['norm_sentiment'] = df_tweets['sentiment'].apply(lambda x: (x - min_val)/(max_val - min_val))
<ipython-input-20-e11ddfafc2ca>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['binary_sentiment'] = df_tweets['norm_sentiment'].apply(binarize)


,id,text,sentiment,tweet,norm_sentiment,binary_sentiment
0,1.342326e+18,we wish a grand success for supreme hero for r...,0.278125,We wish a grand success for Supreme Hero @IamS...,0.639062,1
1,1.342326e+18,sorry to disappoint but I ve not make a dollar...,-0.750000,"Sorry to disappoint, but I’ve not made a dolla...",0.125000,0
2,1.342326e+18,this be wild the who change the definition of ...,0.237500,This is wild.\nThe WHO changed the definition ...,0.618750,1
3,1.342326e+18,patch she be with us year this week we all wis...,0.600000,@mayawiley Patches. She is with us 2 years thi...,0.800000,1
4,1.342326e+18,docsend ceo peer into a post pandemic future,0.000000,DocSend CEO peers into a post-pandemic future ...,0.500000,0


In [ ]:
# add multiclass to the dataset
NEGATIVE_BOUND = 0.4
POSITIVE_BOUND = 0.6

def multiclass_sentiment(val):
    if val >= POSITIVE_BOUND:
        return 2
    elif val <= NEGATIVE_BOUND:
        return 0
    else:
        return 1

df_tweets['multiclass_sentiment'] = df_tweets['norm_sentiment'].apply(multiclass_sentiment)
df_tweets.head(3)

<ipython-input-21-1d87527876ac>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['multiclass_sentiment'] = df_tweets['norm_sentiment'].apply(multiclass_sentiment)


,id,text,sentiment,tweet,norm_sentiment,binary_sentiment,multiclass_sentiment
0,1.342326e+18,we wish a grand success for supreme hero for r...,0.278125,We wish a grand success for Supreme Hero @IamS...,0.639062,1,2
1,1.342326e+18,sorry to disappoint but I ve not make a dollar...,-0.750000,"Sorry to disappoint, but I’ve not made a dolla...",0.125000,0,0
2,1.342326e+18,this be wild the who change the definition of ...,0.237500,This is wild.\nThe WHO changed the definition ...,0.618750,1,2


In [ ]:
df_tweets.to_csv(os.path.join(data_root ,'preprocessed_tweets.csv'))